# **This file performs below operations** -


***ER Modeling - timeframe (2019-2022)***
```
1.   Load 'P20_ER.xlsx' - 30-min observations
2.   This already contains all required data columns
3.   If needed perform aggregation at a level (hourly, daily ?)
4.   Probably train on 2019, 2020, 2021 data and test on 2022?
5.   Save the results to pickle files

```





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from os import walk
from time import sleep
from tqdm import tqdm
import pickle
import string
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import plotly.express as px
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
main_path = '/content/drive/MyDrive/DS for Ag - Alfalfa/Fall 2023'
data_path = os.path.join(main_path, 'Data')
code_path = os.path.join(main_path, 'Code')

In [ ]:
p20_evi_lswi_df = pd.read_excel('/content/drive/MyDrive/DS for Ag - Alfalfa/Fall 2023/Data/ER/P20_ER.xlsx')

In [ ]:
p20_evi_lswi_df

,Year,DoY,Hour,Rg,Tair,Tsoil,rH,VPD,Ustar,SWC,GPP,NEE,ET,ER
0,2019,91,0.5,0.0,2.57,9.73,73.91,-9999.00,0.05,0.306482,0.55305,0.61,0.000358,1.16305
1,2019,91,1.0,0.0,2.56,9.39,73.33,-9999.00,0.03,0.306471,0.55226,0.29,0.000329,0.84226
2,2019,91,1.5,0.0,1.88,9.12,76.41,-9999.00,0.02,0.306262,0.49811,1.35,0.000964,1.84811
3,2019,91,2.0,0.0,1.78,8.83,74.55,-9999.00,0.03,0.306530,0.49012,0.63,0.000467,1.12012
4,2019,91,2.5,0.0,1.84,8.57,68.79,-9999.00,0.03,0.305710,0.49492,0.76,0.000832,1.25492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65802,2022,365,21.5,0.0,7.42,6.25,74.29,0.32,0.12,0.283904,NaN,NaN,0.008344,NaN
65803,2022,365,22.0,0.0,6.96,6.15,71.48,1.37,0.10,0.283348,0.00000,1.36,0.000818,1.36000
65804,2022,365,22.5,0.0,8.19,6.13,69.82,0.37,0.27,0.283078,NaN,NaN,0.025608,NaN
65805,2022,365,23.0,0.0,6.82,6.10,81.70,0.00,0.14,0.282934,0.00000,8.95,-0.007833,8.95000


In [ ]:
p20_evi_lswi_df['Timestamp'] = pd.to_datetime(p20_evi_lswi_df['Year'].astype(str) + p20_evi_lswi_df['DoY'].astype(str), format='%Y%j') + pd.to_timedelta(p20_evi_lswi_df['Hour'], unit='h')


In [ ]:
p20_evi_lswi_df.drop(['Year', 'DoY', 'Hour'], axis=1, inplace = True)

In [ ]:
p20_evi_lswi_df = p20_evi_lswi_df[['Timestamp', 'Rg',	'Tair',	'Tsoil',	'rH',	'VPD','Ustar',	'SWC',	'GPP',	'NEE',	'ET',	'ER']]


In [ ]:
p20_evi_lswi_df.rename(columns={'Timestamp': 'Date'}, inplace=True)

In [ ]:
p20_evi_lswi_df.set_index('Date', inplace=True)

In [ ]:
p20_evi_lswi_df

,Rg,Tair,Tsoil,rH,VPD,Ustar,SWC,GPP,NEE,ET,ER
Date,,,,,,,,,,,
2019-04-01 00:30:00,0.0,2.57,9.73,73.91,-9999.00,0.05,0.306482,0.55305,0.61,0.000358,1.16305
2019-04-01 01:00:00,0.0,2.56,9.39,73.33,-9999.00,0.03,0.306471,0.55226,0.29,0.000329,0.84226
2019-04-01 01:30:00,0.0,1.88,9.12,76.41,-9999.00,0.02,0.306262,0.49811,1.35,0.000964,1.84811
2019-04-01 02:00:00,0.0,1.78,8.83,74.55,-9999.00,0.03,0.306530,0.49012,0.63,0.000467,1.12012
2019-04-01 02:30:00,0.0,1.84,8.57,68.79,-9999.00,0.03,0.305710,0.49492,0.76,0.000832,1.25492
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 21:30:00,0.0,7.42,6.25,74.29,0.32,0.12,0.283904,NaN,NaN,0.008344,NaN
2022-12-31 22:00:00,0.0,6.96,6.15,71.48,1.37,0.10,0.283348,0.00000,1.36,0.000818,1.36000
2022-12-31 22:30:00,0.0,8.19,6.13,69.82,0.37,0.27,0.283078,NaN,NaN,0.025608,NaN


In [ ]:
p20_evi_lswi_df.isnull().sum()/len(p20_evi_lswi_df)

Rg       0.000000
Tair     0.000000
Tsoil    0.000000
rH       0.000000
VPD      0.000000
Ustar    0.000000
SWC      0.302384
GPP      0.509353
NEE      0.509353
ET       0.500661
ER       0.509353
dtype: float64

In [ ]:
# Dropping rows with missing values based on the 'ER' column
p20_evi_lswi_df = p20_evi_lswi_df.dropna(subset=['ER'])

In [ ]:
p20_evi_lswi_df

,Rg,Tair,Tsoil,rH,VPD,Ustar,SWC,GPP,NEE,ET,ER
Date,,,,,,,,,,,
2019-04-01 00:30:00,0.0,2.57,9.73,73.91,-9999.00,0.05,0.306482,0.55305,0.61,0.000358,1.16305
2019-04-01 01:00:00,0.0,2.56,9.39,73.33,-9999.00,0.03,0.306471,0.55226,0.29,0.000329,0.84226
2019-04-01 01:30:00,0.0,1.88,9.12,76.41,-9999.00,0.02,0.306262,0.49811,1.35,0.000964,1.84811
2019-04-01 02:00:00,0.0,1.78,8.83,74.55,-9999.00,0.03,0.306530,0.49012,0.63,0.000467,1.12012
2019-04-01 02:30:00,0.0,1.84,8.57,68.79,-9999.00,0.03,0.305710,0.49492,0.76,0.000832,1.25492
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:30:00,0.0,7.50,6.58,75.85,0.00,0.07,0.284217,0.00000,1.59,0.000767,1.59000
2022-12-31 20:00:00,0.0,7.21,6.47,71.93,0.91,0.08,0.284198,0.00000,0.06,NaN,0.06000
2022-12-31 20:30:00,0.0,7.97,6.41,71.28,1.41,0.10,0.284302,0.00000,1.69,0.001927,1.69000


In [ ]:
# Dropping rows with missing values based on the 'ER' column
p20_evi_lswi_df = p20_evi_lswi_df.dropna(subset=['ET'])

In [ ]:
p20_evi_lswi_df.isnull().sum()/len(p20_evi_lswi_df)

Rg       0.000000
Tair     0.000000
Tsoil    0.000000
rH       0.000000
VPD      0.000000
Ustar    0.000000
SWC      0.050222
GPP      0.000000
NEE      0.000000
ET       0.000000
ER       0.000000
dtype: float64

In [ ]:
pickle_dir = os.path.join(data_path, 'pickle files')
# save the combined alfalfa data to a pickle file
pickle_dir = os.path.join(data_path, 'pickle files')
if os.path.exists(pickle_dir) == False:
  os.mkdir(pickle_dir)

In [ ]:
file_name = 'p20_comp_df.pkl'
file_path = os.path.join(pickle_dir, file_name)
# save/dump to pickle
with open(file_path, 'wb') as handle:
    pickle.dump(p20_evi_lswi_df, handle, protocol=pickle.HIGHEST_PROTOCOL)